In [1]:
import pandas as pd
import numpy as np
import glob

In [23]:
list_of_past_trials = glob.glob('./../Past_Trials/csv_results/'+'*.csv')
all_trials = pd.DataFrame()
for trial in list_of_past_trials:
    print(trial)
    trial_df = pd.read_csv(trial)
    all_trials = pd.concat([all_trials,trial_df],axis=0)
cleaned_all_trials = all_trials.loc[:, ["path_location", "Kernel", "BIC"]]
cleaned_all_trials.to_csv("./../Past_Trials/cleaned_all_trials.csv", index=False)
cleaned_all_trials

./../Past_Trials/csv_results/full_path_history_2023-05-29-19:01:58_cleaned.csv
./../Past_Trials/csv_results/full_path_history_2023-05-30-15:22:27_cleaned.csv
./../Past_Trials/csv_results/full_path_history_2023-05-30-23:04:30_cleaned.csv
./../Past_Trials/csv_results/Starting_with_RBF.csv


,path_location,Kernel,BIC
0,"(0, 0)",'AR2',-2272.004639
1,"(1, 0)",'AR2+RQ+RBF',-2294.232910
2,"(2, 0)",'AR2+RQ*AR2+RBF',-2288.347900
3,"(0, 1)",'AR2+Mat_2.5',-2309.920654
4,"(1, 1)",'AR2+RQ+Mat_2.5',NaN
...,...,...,...
110,"(0, 22)",'RBF*Per_Month',1889.415039
111,"(1, 22)",'RBF+AR2*Per_Month',290.225006
112,"(2, 22)",'RBF+AR2*Per_Year*Per_Month',-2197.544922
113,"(3, 22)",'RBF+AR2*Per_Year*RBF*Per_Month',-1335.378174
